In [ ]:
// Constraints
CREATE CONSTRAINT unique_complaint ON (n:Complaint) ASSERT n.complaint_id IS UNIQUE

CREATE CONSTRAINT unique_precinct ON (n:PrecinctName) ASSERT n.precinct IS UNIQUE

CREATE CONSTRAINT unique_location ON (n:Location) ASSERT n.location_id IS UNIQUE

CREATE CONSTRAINT unique_general_offense ON (n:GeneralOffense) ASSERT n.offense IS UNIQUE

CREATE CONSTRAINT unique_specific_offense ON (n:SpecificOffense) ASSERT n.specific_offense IS UNIQUE

In [ ]:
// Load Complaint nodes
LOAD CSV WITH HEADERS FROM "file:///complaints.csv" as row
CREATE (c:Complaint {
    complaint_id: row.complaint_id,
    location_of_complaint: row.location_of_complaint,
    premises_type: row.premises_type,
    complaint_date: row.complaint_date
})

In [ ]:
// Load PrecinctName nodes and create REPORTED relationship with Complaint
LOAD CSV WITH HEADERS FROM "file:///precincts.csv" as row
MERGE (p:PrecinctName {
    precinct: row.precinct,
    patrol_boro: row.patrol_boro,
    area: row.area})
WITH row, p
MATCH(c:Complaint {complaint_id: row.complaint_id})
MERGE (p)-[:REPORTED]->(c)

In [ ]:
// Load Location node and create LOCATED_AT relationship with Complaint
LOAD CSV WITH HEADERS FROM "file:///locations.csv" as row
MERGE (l:Location {
location: point({
latitude:toFloat(row.latitude), 
longitude:toFloat(row.longitude)}),
latitude: row.latitude,
longitude: row.longitude,
location_id: row.location_id,
boro_name: row.boro_name
})
WITH row, l
MATCH(c:Complaint {complaint_id: row.complaint_id})
MERGE (c)-[:LOCATED_AT]->(l)

In [ ]:
LOAD CSV WITH HEADERS FROM "file:///locations_near.csv" AS row
//look up the two nodes we want to connect up
MATCH (l1:Location {location_id:row.in_location_id}),(l2:Location {location_id:row.near_location_id})
//now create a relationship between them
CREATE (l1)-[:LOCATED_NEARBY {
near_distance: row.near_dist,
near_rank: row.near_rank
}]->(l2);

In [ ]:
// Load Offense:GeneralOffense node and create COMMITTED_OFFENSE relationship with Complaint
LOAD CSV WITH HEADERS FROM "file:///general_offenses.csv" as row
MERGE (o:Offense:GeneralOffense {
    offense: row.offense,
    offense_id: row.offense_id

})
WITH row, o
MATCH(c:Complaint {complaint_id: row.complaint_id})
MERGE (c)-[:COMMITTED_OFFENSE]->(o)

In [ ]:
LOAD CSV WITH HEADERS FROM "file:///specific_offenses.csv" as row
MERGE (s:Offense:SpecificOffense {
    offense: row.specific_offense,
    offense_id: row.specific_offense_id

})
WITH row, s
MATCH(o:Offense:GeneralOffense {offense_id: row.offense_id})
MERGE (o)-[:CLASSIFIED_AS]->(s)